In [144]:
import random
import queue
import concurrent.futures
import math
import numpy as np
from functools import reduce
from threading import Barrier

def share(secret, client_num):
    """
    Producing share based on the number of clients.
    """
    shares = [0] * client_num
    for i in range(client_num - 1):
        shares[i] = random.randint(1, 6)
    shares[client_num - 1] = secret - sum(shares[:client_num - 1])
    return shares

def generate_shares(binary_list, n):
    shares = []
    current_share = binary_list
    for _ in range(n-1):
        new_share = [random.randint(0, 1) for _ in range(8)]
        current_share = [a ^ b for a, b in zip(current_share, new_share)]
        shares.append(new_share)
    shares.append(current_share)
    return shares

def reconstruct(shares):
    result = [0] * 8
    for share in shares:
        result = [a ^ b for a, b in zip(result, share)]
    return result

def toBinary(A):
    neg = False
    if A < 0:
        neg = True
        A = -A
    B = []
    while A > 0:
        B.append(A % 2)
        A = math.floor(A / 2)
    B.append(0)
    if neg:
        c = 0
        for i in range(len(B)):
            if c:
                B[i] = 1 - B[i]
            else:
                c = B[i] == 1
    B.reverse()
    return B

def fromBinary(B):
    if B[0] == 0:
        return reduce(lambda a, b: 2*a + b, B)
    else:
        return -fromBinary([1 - b for b in B]) - 1

def CP(i, x, y, U, V, W, r, input_queue, output_queue, barrier):
    """
    The main function each party executes separately.
    """
    
    def A2B(s):
        return s % 2
    
    def B2A(i, s):
        r_2 = A2B(r)
        c = [bit ^ r_2 for bit in s]

        for q in output_queue:
            q.put(c)
        
        for _ in range(len(output_queue)):
            c = [bit1 ^ bit2 for (bit1, bit2) in zip(c, input_queue.get())]
        
        if i == 0:
            return [bit + r - 2 * bit * r for bit in c]
        else:
            return [r - 2 * bit * r for bit in c]

    def mul_number(i, s, d):
        """
        Multiplication function based on Beaver triple.
        """
        D = s - U
        E = d - V

        # Sending and receiving intermediate results
        for q in output_queue:
            q.put((D, E))

        DE_values = [(D, E)] + [input_queue.get() for _ in range(len(output_queue))]

        D_sum = sum(D for D, E in DE_values)
        E_sum = sum(E for D, E in DE_values)

        Z = W + (D_sum * V) + (U * E_sum)

        if i == 0:
            Z += (D_sum * E_sum)
        return Z
    
    def o1(d2, d1):
        """
        Carry propagation operator o.
        Computes a new (p, g) pair from two input pairs:
        (p, g) = (p2, g2) o (p1, g1) = (p1 & p2, g2 | (p2 & g1))
        """
        (p2, g2) = d2
        (p1, g1) = d1
        return (p1 & p2, g2 | (p2 & g1))

    def o(i, d2, d1):
        (p2, g2) = d2
        (p1, g1) = d1
        return (mul_number(i, p1, p2), g2 + mul_number(i, p2, g1))
    
    def PreOpL1(A):
        i = 1
        j = 2
        while j < len(A):
            for k in range(math.floor(len(A) / j)):
                p = i + k * j
                for q in range(i):
                    A[p+q] = o(A[p], A[p+q])
            i = j
            j *= 2
        return A
    
    def PreOpL(i, A):
        k = len(A)
        kh = math.floor(k / 2)

        P = A
        P[0] = A[0]

        if k > 1:
            U = [0] * kh
            for j in range(1, 1+kh):
                U[j-1] = o(i, A[2*j - 1], A[2*j - 2])
            V = PreOpL(i, U)
            for j in range(1, 1+kh):
                P[2*j - 1] = V[j - 1]
            for j in range(2, 1+kh):
                P[2*j - 2] = o(i, A[2*j - 2], V[j - 2])
        
        return P

    def PreOpC(D):
        """
        Computes the carry bits from the input (p, g) pairs.
        The input consists of a (p, g) pair for each pair of bits to add.
        Each resulting pair is computed as (Pi, Gi) = (pi, gi) o (P[i-1], G[i-1])
        with (P0, G0) = (p0, g0).
        From each resulting pair, the carry bit can be obtained with ci = Gi.
        """
        p = D[0]
        P = [p]
        for i in range(1,len(D)):
            p = o(D[i], p)
            P.append(p)
        return P
    
    def addbitwise(i, A, B):
        """
        Performs bitwise addition on the given bit strings.
        The input parameters A and B are arrays of integers representing bits.
        The leftmost bits (A[0] and B[0]) represent the most significant bits.
        The output is a single array of bits representing the value A+B.
        """
        
        # Set the lengths of the bit strings to the next power of 2
        k = max(len(A),len(B))

        # Padding by extension of most significant bit
        A = [A[0]] * (k - len(A)) + A
        B = [B[0]] * (k - len(B)) + B

        A.reverse()
        B.reverse()

        # Calculate carry bits
        D = [(a + b - 2 * ab, ab) for (a, b) in zip(A, B) if (ab := mul_number(i, a, b))]
        C = PreOpL(i, D)

        # Perform addition with carry bits
        S = [A[0] + B[0] - 2*C[0][1]]
        for i in range(1, k):
            S.append(A[i] + B[i] + C[i-1][1] - 2*C[i][1])
        S.reverse()

        # Debug output
        A.reverse()
        B.reverse()
        C.reverse()
        C = [c for (_, c) in C]
        #print("A=", A)
        #print("B=", B)
        #print("S=", S, "cbit=", C[0])

        return S
    
    def CarryOutAux(D):
        k = len(D)
        if k > 1:
            U = []
            for i in range(1, 1 + math.floor(k/2)):
                U.append(o(D[2*i - 1], D[2*i - 2]))
            return CarryOutAux(U)
        else:
            return D[0]
    
    def CarryOutLin(D):
        return PreOpC(D)[len(D)-1]
    
    def CarryOutCin(A, B, c):
        pos = A[0] + B[0] == 0
        k = len(A)
        if k > len(B):
            B = [B[0] * (k - len(B))] + B
        elif k < len(B):
            A = [A[0] * (len(B) - k)] + A
            k = len(B)
        if pos:
            k -= 1
            A = A[1:]
            B = B[1:]

        A.reverse()
        B.reverse()

        # Calculate carry bits
        D = [(a + b - 2*a*b, a * b) for (a, b) in zip(A, B)]
        (p, g) = D[0]
        D[0] = (p, g + c*p)
        (_, g) = CarryOutLin(D)
        return g
    
    def BitLT(A, B):
        B = [1 - b for b in B]
        return 1 - CarryOutCin(A, B, 1)

    # Share x and y over field F
    xF = B2A(i, x)
    yF = B2A(i, y)
    print(" ")
    #print("B2A", x, B2A(i, x))

    print("add", addbitwise(i, xF, yF))
    res  = addbitwise(i, xF, yF)
    #cbit = CarryOutCin          (toBinary(x), toBinary(y), 0)
    #lt   = BitLT                (toBinary(x), toBinary(y))
    #print(x,"+",y,"=",res,"c=",cbit,"lt=",lt)
    return res

def SPDZ_prepare(beta1, beta2, group_size):
    """
    Function to produce Beaver triple shares.
    """
    U, V = random.randint(3, 6), random.randint(3, 6)
    W = U * V
    r = random.randint(0, 1)

    x_shares = generate_shares(beta1, n)
    y_shares = generate_shares(beta2, n)
    return share(U, group_size), share(V, group_size), share(W, group_size), x_shares, y_shares, share(r, group_size)

def SPDZ_execute(preparation_data, group_size):
    """
    Function to execute the SPDZ protocol.
    """
    with concurrent.futures.ThreadPoolExecutor() as executor:
        barrier = Barrier(group_size)
        U_shares, V_shares, W_shares, x_shares, y_shares, r_shares = preparation_data
        queues = [queue.Queue() for _ in range(group_size)]
        futures = [
            executor.submit(
                CP, j, x_shares[j], y_shares[j], U_shares[j], V_shares[j], W_shares[j], r_shares[j],
                queues[j], [queues[k] for k in range(group_size) if k != j], barrier
            ) for j in range(group_size)
        ]

        group_results = [f.result() for f in futures]
        print("Final results share of each party:", group_results)
        sum_total = np.sum(group_results, axis=0).tolist()
        print("Final Result:", sum_total)

# Main execution
if __name__ == "__main__":
    n = 5  # Number of parties
    input1 = [0, 0, 0, 1, 0, 1, 0, 0] # 20
    input2 = [0, 0, 0, 0, 1, 0, 1, 0] # 10

    preparation_data = SPDZ_prepare(input1, input2, n)
    SPDZ_execute(preparation_data, n)


  
 
 

 
add [-318, -28, -218, -50, -200, -100, -142, -86]
add [178, 28, 128, 24, 102, 49, 72, 58]
add [12, -3, 6, 2, 17, 5, 14, 0]
add [36, -4, 22, 8, 26, 15, 18, 4]
add [92, 7, 62, 17, 56, 32, 39, 24]
Final results share of each party: [[12, -3, 6, 2, 17, 5, 14, 0], [92, 7, 62, 17, 56, 32, 39, 24], [36, -4, 22, 8, 26, 15, 18, 4], [178, 28, 128, 24, 102, 49, 72, 58], [-318, -28, -218, -50, -200, -100, -142, -86]]
Final Result: [0, 0, 0, 1, 1, 1, 1, 0]
